# Example – Pool Fire

## Introduction

This example illustrates the setup and analysis of a minimalistic FDS simulation. A simple pool fire with a fixed heat release rate is computed. 

## Setup

The scenario is a pool fire with a heat release rate of $\small\sf 1~MW$ and a surface area of $\small\sf 1~m^2$ is located centraly at the floor, i.e. $\small\sf z=0~m$. The extension of the compuatational domain is 

$$\small\sf [-2.5~m, 2.5~m] \times [-2.5~m, 2.5~m] \times [0~m, 10~m]$$ 

with a grid resolution of $\small\sf 0.1~m$.

## FDS Input File

```{literalinclude} ../../../../data/pool_fire/basic/pool_fire_basic.fds
```

**TODO:** Description of the input file. 

## SMV Visualisation

:::{figure-md} fig-pool-fire-smv-plain

<img src="figs/pool_fire_smv_plain.png" width="60%">

SMV visualisation of the geometry. The surface patch `BURNER`, which has a `HRRPUA` defined, is colored redish.  
:::

:::{figure-md} fig-pool-fire-smv-grid

<img src="figs/pool_fire_smv_grid.png" width="60%">

SMV visualisation of the grid in the y-z-plane.
:::

:::{figure-md} fig-pool-fire-smv-slcf

<img src="figs/pool_fire_smv_slcf.png" width="60%">

SMV visualisation of the instanteneous ($\sf t=19.63~s$) temperature values in the y-z-slice $\sf y=0~m$.
:::

## Analysis

In [75]:
import fdsreader as fds
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 150

root = '../../../../'
data_root = root + 'data/pool_fire/basic/rundir'

sim = fds.Simulation(data_root)

In [76]:
slice = sim.slices[0]
time = 19.63
time_index = slice.get_nearest_timestep(time)

fig = plt.figure(figsize=(10,5))
plt.imshow(slice[0].data[time_index].T, origin='lower', 
           extent=slice[0].extent.as_list(), cmap='Oranges')
plt.xlabel("x / m")
plt.ylabel("z / m")
plt.colorbar(label='Temperature / C')
plt.savefig('figs/temperature.svg', bbox_inches='tight')
plt.close()

**TODO:** alternative representation with Python.

:::{figure-md} fig-pool-fire-temp-slice

<img src="figs/temperature.svg" width="60%">

Visualisation of the instanteneous ($\sf t=19.63~s$) temperature values in the y-z-slice $\sf y=0~m$.
:::

In [74]:
plt.plot(sim.hrr["Time"], sim.hrr["HRR"], label='simulation')
t_min_max = [sim.hrr["Time"][0], sim.hrr["Time"][-1]]
hrr_prescribed = 1000
plt.plot(t_min_max, [hrr_prescribed, hrr_prescribed], label='prescribed')
plt.legend()
plt.xlabel('time / s')
plt.ylabel('heat release rate / kW')
plt.savefig('figs/pool_fire_hrr.svg', bbox_inches='tight')
plt.close()

**TODO:** Discusson of the HRR curve in {numref}`fig-pool-fire-hrr`

:::{figure-md} fig-pool-fire-hrr

<img src="figs/pool_fire_hrr.svg" width="80%">

Prescribed and modelled heat release rate as a function of time.
:::

## Task I – Basics

The goal of this task is to become familiar with basic functions. Use the above listed input file and extend it.

```{tip} When adding new diagnostics or other elements, it is useful to reduce the computing time to allow for a fast turn-over-time. This way debugging or adjusting your setup will be more efficient. For pure geometrical checks, the simulaiton time can also be set to zero.  
```

```{tip} Table 21.3 in the [FDS User's Guide](https://github.com/firemodels/fds/releases/download/FDS6.7.5/FDS_User_Guide.pdf) provides a summary of frequently used output quantities.  
```

**Useful steps to consider first:**

1. Reduce the computing time to $\small\sf t_{end} = 5~s$.
1. Add a z-velocity slice at $\small\sf y=0~m$.
1. Increase the heat release rate to $\small\sf 5~MW$.

**Task:**

Compare the z-velocity slices for heat release rates of $\small\sf 100~kW$, $\small\sf 1~MW$ and $\small\sf 10~MW$. Create snapshots for three points in time, e.g. $\small\sf 10~s$, $\small\sf 75~s$ and $\small\sf 150~s$.

## Task II – Mean Profiles

* v_z profile along z-axis as function of HRR
* temperature stratification 
* DEVC, individual, line
* analysis of SLCF
* mean + stddev

## Task III – Soot Production

* add a soot yield to the combustion reaction
* describe the development of the smoke layer

## Task IV – Time-dependent Heat Release Rate

* use the RAMP function to define a time dependent function
* open boundaries

## Task V – Grid Convergence

* pick an observable, e.g. velocity profile
* increase the grid resolution by factors of 0.5, 2 and 4
* monitor the computing time, see .out file
* use a parallel computer for factors of 8 and 16, including multiple meshes 